## Layers and hodoscopes


The default *layer* class `PanelDetectorLayer` in TomOpt uses multiple *panels* `DetectorPanel` to record muon position.

`PanelDetectorLayer` inherits from the `AbsDetectorLayer` and **MUST** provide the following **methods**:

 - `forward`: The forward method to propagate the muons. It is a central aspect of `nn.Module`.
 - `get_cost`
 - `conform_detector`
  - `assign_budget`

and the following **features**:

 - `pos`: the position of the layer, either `above` or `bellow`. 
 - `lw`: the length and width of the layer.
 - `z`: the z position of the top of gthe layer. 
 - `size`: the heigth of the layer such that z - size is the bottom of the layer.
 - `device`: torch.device.

In [1]:
from torch import Tensor
# from torch import nn
from volume.hodoscope import Hodoscope
from volume.hodoscopelayer import HodoscopeDetectorLayer

hod1 = Hodoscope(init_xyz= [.5, .5, 1.], 
                 init_xyz_span = [1., 1., .3], 
                 xyz_gap = [.1, .1, .1],
                 panel_type = 'DetectorPanel')

hod2 = Hodoscope(smooth=0.5,
                 init_xyz = [0.25, 0.25, 0.6], 
                 init_xyz_span = [1., 1., .3], 
                 xyz_gap = [.1, .1, .1])


hods = [hod1, hod2]

hod_detector_above = HodoscopeDetectorLayer(pos = 'above', lw = Tensor([1,1]), z = 1., size = 0.5, 
                                            hodoscopes = hods)

print(hod2.get_cost())

tensor(1.9200)


In [4]:
print(hod1.xyz_max)

[1.5, 1.5, 1.3]


In [ ]:
# Test clamping feature

import torch

print("PRIMA")
for i, p in hod_detector_above.yield_zordered_panels():
    print("xyz panels:", i, p.xy, p.z)
    if i ==2:
        p.z.data = torch.tensor(45.)

print("MODIFICATO")
for i, p in hod_detector_above.yield_zordered_panels():
    print("xyz panels:", i, p.xy, p.z)
    
    
hod_detector_above.conform_detector()

print("CLAMPEADO")
for hod in hod_detector_above.hodoscopes:
    print("\t xyz hodoscope:", hod.xy, hod.z)
    for i, p in hod_detector_above.yield_zordered_panels():
        print("\t\t xyz panels:", i, p.xy, p.z)


PRIMA
xyz panels: 0 Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True) Parameter containing:
tensor([0.9000], requires_grad=True)
xyz panels: 1 Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True) Parameter containing:
tensor([0.8500], requires_grad=True)
xyz panels: 2 Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True) Parameter containing:
tensor([0.8000], requires_grad=True)
xyz panels: 3 Parameter containing:
tensor([0.2500, 0.2500], requires_grad=True) Parameter containing:
tensor([0.5000], requires_grad=True)
xyz panels: 4 Parameter containing:
tensor([0.2500, 0.2500], requires_grad=True) Parameter containing:
tensor([0.4500], requires_grad=True)
xyz panels: 5 Parameter containing:
tensor([0.2500, 0.2500], requires_grad=True) Parameter containing:
tensor([0.4000], requires_grad=True)
MODIFICATO
xyz panels: 2 Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True) Parameter containing:
tensor(45., requires_grad=True)
xyz pane

AttributeError: 'Hodoscope' object has no attribute 'clamp_params'